In [1]:
import glob
import json
import pprint

In [7]:
def check_doc(doc:dict):
    
    data = doc["dataset"]
    q = doc["question"]
    a = doc["answers"]
    pos = len(doc["positive_ctxs"])
    neg = len(doc["negative_ctxs"])
    h_neg = len(doc["hard_negative_ctxs"])
    
    print(f"Dataset: {data} | Q: {q} | A: {a}\npos:{pos} neg:{neg} hard neg:{h_neg}")

In [2]:
k = ['dt', 'q', 'a', 'p', 'n', 'hn']
dk = ['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs']
doc_keys = dict(zip(k, dk))

In [3]:
data_path="./data/retriever/*train.json"

In [4]:
irdata_list = glob.glob(data_path)
irdata_list

['./data/retriever/nq-train.json',
 './data/retriever/trivia-train.json',
 './data/retriever/nq-adv-hn-train.json',
 './data/retriever/curatedtrec-train.json',
 './data/retriever/webq-train.json',
 './data/retriever/squad1-train.json']

In [5]:
with open( irdata_list[0], "r",encoding='utf8') as fr:
    docs = json.load(fr)

In [6]:
print(f"{docs[0].keys()}\n{len(docs)}")

dict_keys(['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs'])
58880


#### 키값 내용
  
dataset : 데이터 셋 정보.위키 triviz, trec, squad.  
question: 질문 문장 하나  
answers: mrc 최종 결과. 이하 '답'이라 하겠음  
positive_ctxs: 답이 포함되어 있고, bm25 상위 랭크에 걸리는 결과들  
negative_ctxs: random인 것 같음. 검색 스코어 0  
hard_negative_ctxs: 검색 스코어가 높은데 답은 없는 검색 결과  

In [8]:
def get_datatext(idx, doc_key):
    pprint.pprint(docs[idx][doc_key])

title_score는 rank1 만 1 이고, 나머진 0. 뭘 기준으로 하는지 모르겠음. 
논문에서는 모든 passage에 제목을 [SEP] 토큰으로 이어붙였다고 함.

In [9]:
idx = 19

In [28]:
check_doc(docs[idx])
get_datatext(idx, "dataset")

Dataset: nq_train_psgs_w100 | Q: when did roller derby first appear in the press | A: ['1922']
pos:2 neg:50 hard neg:99
'nq_train_psgs_w100'


In [27]:
doc_key = doc_keys["q"]
print(docs[idx][doc_key])
doc_key = doc_keys["p"]
print(len(docs[idx][doc_key]))
pprint.pprint(docs[idx][doc_key][0])
print("-"*11)
pprint.pprint(docs[idx][doc_key][1])
#check_doc(docs[idx])
#(idx, "positive_ctxs")

when did roller derby first appear in the press
2
{'passage_id': '11105240',
 'score': 1000,
 'text': 'three decades. Among these races was an 8.5-mile roller marathon '
         'organized in 1908 by a group of Chicago rink owners, a 24-hour '
         'endurance championship held in Milwaukee in 1913, and a 24-hour '
         'banked track race held at Madison Square Garden in 1914. "The New '
         'York Times" noted that the crowd at Madison Square Garden enjoyed '
         "the sudden sprints and spills during the event's preliminary races. "
         'The term "derby", meaning a race or multi-race event, appeared in '
         'the press as early as 1922, when the "Chicago Tribune" announced and '
         'reported on the results of two "roller derby" events held that year.',
 'title': 'History of roller derby',
 'title_score': 1}
-----------
{'passage_id': '11105240',
 'score': 13.821991,
 'text': 'three decades. Among these races was an 8.5-mile roller marathon '
         '

#### score 1000은 무엇인가?

위 예시를 보면(nq_train, index 19 문서)  
완전 동일한 title, text 인데 title_score, score가 서로 다름(1, 1000 : 0, 13.82)  
아마 기존 데이터셋 정답인 경우와 아닌 경우로 나뉘는거 같음

문서마다 pos, neg, hard neg 수가 다다름. 배치 생성시 instance를 다른 조합으로 여러개 만드나? 싶음  

* 논문에서는 q-passgae 쌍 하나가 instance 하나. 배치 안에 다른 답을 가지고 in-batch negative sampling을 함.
* q 하나에 여러 ctxs 들은 한번에 같이 쓰는건 아닐거 같고 그때 그떄 한쌍을 만드는게 아닐까? 하는 중

#### Tokenizer

In [9]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
print(tokenizer.tokenize(text))
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)


['replace', 'me', 'by', 'any', 'text', 'you', "'", 'd', 'like', '.']


In [15]:
text = docs[idx][doc_key][1]["text"]
# whitespace 기준 100
print(len(text.split(" ")))
# sub-word 기준 max len은 256. 코드에 기본값이 그렇게 들어가 있음
print(len(tokenizer.tokenize(text)))

100
132


In [3]:
from transformers import ElectraTokenizer,ElectraModel
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
#tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")
#tokenizer.convert_tokens_to_ids(['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]'])


/home/hjb/miniconda3/envs/rerank/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
text = docs[idx][doc_key][1]["text"]
# whitespace 기준 100
#print(len(text.split(" ")))
# sub-word 기준 max len은 256. 코드에 기본값이 그렇게 들어가 있음
print(len(tokenizer.tokenize(text)))

241


#### 학습 실행 command

* zsh 버전  
```python train_dense_encoder.py --train_datasets=\[nq_train\] dev_datasets=\[nq_dev\] train=biencoder_local```